In [67]:
import pymongo
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime

### Perfroming Hashtag analysis on the DATA of Individual post
##  1) Seprate out numerical data and HashTags/ Caption 
## 2) Will only use Clips as they have reach data 
## 3) will only use the latest instance of a post 

In [6]:
load_dotenv()
MONGO_DB =   os.getenv("MONGO_DB")    
Db = pymongo.MongoClient(MONGO_DB)

In [48]:

posts = Db.Clean.Posts.find(projection = {"_id":0})

In [49]:
df = pd.DataFrame(posts)

In [99]:
df.index

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       11231, 11233, 11234, 11235, 11236, 11237, 11238, 11239, 11240, 11241],
      dtype='int64', length=7611)

In [53]:
df.columns

Index(['media_id', 'type', 'caption', 'hashtags', 'commentsCount',
       'likesCount', 'videoViewCount', 'timestamp', 'ownerId', 'productType',
       'taggedUsers', 'followersCount', 'ER_per', 'Reach_per', 'Likes_per',
       'Cmnts_per', 'LC_ratio'],
      dtype='object')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11242 entries, 0 to 11241
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   media_id        11242 non-null  object        
 1   type            11242 non-null  object        
 2   caption         11242 non-null  object        
 3   hashtags        11242 non-null  object        
 4   commentsCount   11242 non-null  int64         
 5   likesCount      11242 non-null  int64         
 6   videoViewCount  7611 non-null   float64       
 7   timestamp       11242 non-null  datetime64[ns]
 8   ownerId         11242 non-null  object        
 9   productType     7611 non-null   object        
 10  taggedUsers     3341 non-null   object        
 11  followersCount  11242 non-null  int64         
 12  ER_per          11242 non-null  float64       
 13  Reach_per       7611 non-null   float64       
 14  Likes_per       11242 non-null  float64       
 15  Cm

In [59]:
for i in df.columns:
    if df[i].dtype == object:
        if type(df[i][0]) == list:
            continue
    print(f"Unique values for column {i}")
    print(df[i].unique())

Unique values for column media_id
['3357032502832118180' '3357026465022242692' '3356585424787923656' ...
 '3377271032262550134' '3376724566427471684' '3376544424451110496']
Unique values for column type
['Video' 'Sidecar' 'Image']
Unique values for column caption
['#hindu #hinduism #india #hindugod #shiva #hindugods #historicalplace #mahadev #oldplaces #vision #paceful #hinata #arch #myspirit #myself #frescos #hindus #omnamahshivaya #harharmahadev #ganesha #om #hindutemple #spirituality #indialovers #hinduwedding #ganesh #hindustan #hinduismo #indian #love'
 "Don't forget to follow us\n\nLike\nComment\nShare\nTag your friend's\n\n#hindu #hinduism #india #hindugod #shiva #hindugods #historicalplace #mahadev #oldplaces #vision #paceful #hinata #arch #myspirit #myself #frescos #hindus #omnamahshivaya #harharmahadev #ganesha #om #hindutemple #spirituality #indialovers #hinduwedding #ganesh #hindustan #hinduismo #indian"
 "Don't forget to follow us\n\nLike\nComment\nShare\nTag your friend's

In [ ]:
df = df[['media_id', 'type', 'caption', 'hashtags', 'timestamp', 'productType', 'followersCount', 'ER_per', 'Reach_per', 'Likes_per',
       'Cmnts_per', 'LC_ratio']]
df = df[df["type"]=="Video"]

In [63]:
for i in df.columns:
    if df[i].dtype == object:
        if type(df[i][0]) == list:
            continue
    print(f"Unique values for column {i}")
    print(df[i].unique())

Unique values for column media_id
['3357032502832118180' '3357026465022242692' '3356585424787923656' ...
 '3377456023860990240' '3377271032262550134' '3376544424451110496']
Unique values for column type
['Video']
Unique values for column caption
['#hindu #hinduism #india #hindugod #shiva #hindugods #historicalplace #mahadev #oldplaces #vision #paceful #hinata #arch #myspirit #myself #frescos #hindus #omnamahshivaya #harharmahadev #ganesha #om #hindutemple #spirituality #indialovers #hinduwedding #ganesh #hindustan #hinduismo #indian #love'
 "Don't forget to follow us\n\nLike\nComment\nShare\nTag your friend's\n\n#hindu #hinduism #india #hindugod #shiva #hindugods #historicalplace #mahadev #oldplaces #vision #paceful #hinata #arch #myspirit #myself #frescos #hindus #omnamahshivaya #harharmahadev #ganesha #om #hindutemple #spirituality #indialovers #hinduwedding #ganesh #hindustan #hinduismo #indian"
 "Don't forget to follow us\n\nLike\nComment\nShare\nTag your friend's\n\n#hindu #hindui

In [68]:
df["Time_on_air"] =  df["timestamp"] - datetime.today()

C:\Users\rahul\AppData\Local\Temp\ipykernel_1696\2270151253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Time_on_air"] =  df["timestamp"] - datetime.today()


In [73]:
Max_Reach_dic = {}
for i in df.media_id:

    Max_Reach_dic[i] = df[df["media_id"] == i]["Reach_per"].max() 

In [77]:
len(list(Max_Reach_dic.keys())) == len(df["media_id"].unique())

True

In [106]:
len(df["media_id"].unique())

3912

In [103]:
indexs = []
for i,j in zip(Max_Reach_dic.keys(),Max_Reach_dic.values()):

    indexs.append(df[(df["media_id"]==i) & (df["Reach_per"]==j)].index[0])

In [112]:
df

ValueError: ('Lengths must match to compare', (7611,), (3912,))

In [80]:
for i,j in zip(Max_Reach_dic.keys(),Max_Reach_dic.values()):
    print(df.where()

SyntaxError: incomplete input (1734523815.py, line 2)